# Explore Oura User Data with Elaticsearch & Kibana

Oura Documentation for the Developer API: https://cloud.ouraring.com/v2/docs 

In [1]:
! pip install -r requirements.txt
import requests 
import json
from elasticsearch import Elasticsearch, helpers
from getpass import getpass

Generating an auth token for your account: https://cloud.ouraring.com/docs/authentication

In [44]:
auth_token = getpass("auth token: ")

Getting an initial data dump of all the daily activity and sleep history and saving it in json files.

In [43]:
def get_data(type):
  url = 'https://api.ouraring.com/v2/usercollection/' + type
  params={ 
      'start_date': '2021-11-01', 
      'end_date': '2025-01-01' 
  }
  headers = { 
    'Authorization': 'Bearer ' + auth_token 
  }
  response = requests.request('GET', url, headers=headers, params=params) 
  return response.json()["data"]

In [40]:
data = get_data("daily_activity")
with open('oura_data_activity.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [41]:
data = get_data("sleep")
with open('oura_data_sleep.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

## Get the data into elastic

In [2]:
#Connect to the elastic cloud server
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

In [ ]:
index_name = 'oura-history-sleep'

# Create the Elasticsearch index with the specified name (delete if already existing)
if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)
client.indices.create(index=index_name)

with open("oura_data_sleep.json", "r") as f:
    json_data = json.load(f)
    documents = []
    for doc in json_data:
        documents.append(doc)
    load = helpers.bulk(client, documents, index=index_name)

We can now search our data! Let's test it out by starting with a very simple data science experiment, looking a bit deeper at the concept of the sleep score calculated by Oura.

First, we can do a very basic search of getting back the days on which the Oura generated sleep score was the highest

In [6]:
index_name = 'oura-history-sleep'

response = client.search(index = index_name, sort="readiness.score:desc")

for hit in response["hits"]["hits"]:
    print("Day: {day} and sleeping score: {score}".format(day=hit['_source']['day'], score= hit['_source']['readiness']['score']))

Day: 2024-09-03 and sleeping score: 93
Day: 2024-09-13 and sleeping score: 93
Day: 2024-06-28 and sleeping score: 92
Day: 2024-08-31 and sleeping score: 92
Day: 2024-09-10 and sleeping score: 92
Day: 2024-06-23 and sleeping score: 91
Day: 2024-08-16 and sleeping score: 91
Day: 2024-08-19 and sleeping score: 91
Day: 2024-08-21 and sleeping score: 91
Day: 2024-08-23 and sleeping score: 91


We can now use these dates to look a bit closer at the other relevant values that were noted on these data points.

In [37]:
good_days = ["2024-09-03", "2024-09-13", "2024-08-31", "2024-09-10", "2024-06-23"]

for day in good_days:
    query = {
        "match" : {
            "day" : day
        }
    }
    response = client.search(index = index_name, query=query)

    result =  response["hits"]["hits"][0]["_source"]

    print("Slept a total of {time} hours: {ds} hours deep sleep, {rem} hours rem, {awake} hours awake time".format(time=round(result["total_sleep_duration"]/3600,2), ds=round(result["deep_sleep_duration"]/3600, 2), rem=round(result["deep_sleep_duration"]/3600,2), awake=round(result["awake_time"]/3600,2)))
    print("Average heart rate: {rate}.".format(rate=result["average_heart_rate"]))



Slept a total of 10.22 hours: 1.8 hours deep sleep, 1.8 hours rem, 0.99 hours awake time
Average heart rate: 59.375.
Slept a total of 8.98 hours: 1.49 hours deep sleep, 1.49 hours rem, 0.69 hours awake time
Average heart rate: 59.125.
Slept a total of 9.11 hours: 1.5 hours deep sleep, 1.5 hours rem, 0.88 hours awake time
Average heart rate: 58.75.
Slept a total of 9.11 hours: 1.51 hours deep sleep, 1.51 hours rem, 0.93 hours awake time
Average heart rate: 54.5.
Slept a total of 9.41 hours: 1.34 hours deep sleep, 1.34 hours rem, 1.41 hours awake time
Average heart rate: 62.375.


This follows the intutition that longer deep sleep and a relatively lower heart rate result in a better night of sleep. Based on this first glance at the tiny data subsection, we can come up with an very simple search to mimic our own "sleep score".  

As a simple example of this, in the following query we filter on higher deep sleep than 1.5 hours, and a lower heart rate than 60. Finally we can also arrange these results by descending REM sleep time.

To confirm our assumption, we now look at the associated sleep score with the results we get back on our "test dataset".

In [38]:
query = {
    "range" : {
        "deep_sleep_duration" : {
            "gte" : 1.5*3600
        }
    },
    "range" : {
        "average_heart_rate":{
            "lte" : 60
        }
    }
}
response = client.search(index = index_name, query=query, sort="rem_sleep_duration:desc")

for hit in response["hits"]["hits"]:
    print("Day: {day} and sleeping score: {score}".format(day=hit['_source']['day'], score= hit['_source']['readiness']['score']))


Day: 2024-08-15 and sleeping score: 87
Day: 2024-09-08 and sleeping score: 85
Day: 2024-08-19 and sleeping score: 91
Day: 2024-08-18 and sleeping score: 90
Day: 2024-08-20 and sleeping score: 90
Day: 2024-09-09 and sleeping score: 88
Day: 2024-11-09 and sleeping score: 85
Day: 2024-09-03 and sleeping score: 93
Day: 2024-09-10 and sleeping score: 92
Day: 2024-08-14 and sleeping score: 89


This response is consistent with the days we got back on the first search where we filtered by highest sleep score. This confirms our theory that the sleep score formula is probably closely alligned with the variables and filters we naively implemented.

Finally, we can take a look at more of our data in a comprehensive manner to study the same correlations. Here's an example of a Kibana dashboard we built with our sleep data focusing on the same values:

![](kibana.png)